In [23]:
import os
from dotenv import load_dotenv
import warnings

warnings.filterwarnings("ignore")  # Required to suppress warnings from requests

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")

In [24]:
from OptiHPLCHandler import EmpowerHandler


handler = EmpowerHandler(
    project="WebAPI_test",
    address=EMPOWER_API_ADDRESS,
    allow_login_without_context_manager=True,
)

handler.connection.verify = "../ca-certificates.crt"

# Get methods with BSM and QSM pumps as well as PDA and TUV detectors
with handler:
    bsm_pda_method = handler.GetInstrumentMethod("@BSM_PDA_Template")
    qsm_pda_method = handler.GetInstrumentMethod("@QSM_PDA_Template")
    bsm_tuv_method = handler.GetInstrumentMethod("@BSM_TUV_Template")
    qsm_pda_flr_method = handler.GetInstrumentMethod("@QSM_PDA_FLR_Template")
    # bsm_pda_rid_method = handler.GetInstrumentMethod("@BSM_PDA_RI_Template") # not implemented
    bsm_tuv_single_method = handler.GetInstrumentMethod("@BSM_TUV_Single_Template")
    bsm_tuv_dual_method = handler.GetInstrumentMethod("@BSM_TUV_Dual_Template")
    bsm_tuv_off_method = handler.GetInstrumentMethod("@BSM_TUV_Off_Template")
    bsm_pda_on_method = handler.GetInstrumentMethod("@BSM_PDA_ON_Template")

c:\Users\slcb\Desktop\Programming\OptiHPLCHandler\.env\lib\site-packages\urllib3\connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host ***REMOVED***. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


# Convert Gradient

Handlers the interconversion between BSM and QSM methods for both gradient and isocratic flow

In [25]:
# Change Gradient Table
qsm_gradient_table_to_change = [
    {
        "Time": "Initial",
        "Flow": "0.600",
        "CompositionA": "90.0",
        "CompositionB": "10.0",
        "CompositionC": "0.0",
        "CompositionD": "0.0",
        "Curve": "Initial",
    },
    {
        "Time": "1.00",
        "Flow": "0.600",
        "CompositionA": "10.0",
        "CompositionB": "90.0",
        "CompositionC": "0.0",
        "CompositionD": "0.0",
        "Curve": "6",
    },
]

new_bsm_gradient_table = [
    {
        "Time": "Initial",
        "Flow": "0.600",
        "CompositionA": "80.0",
        "CompositionB": "20.0",
        "Curve": "Initial",
    },
    {
        "Time": "1.00",
        "Flow": "0.600",
        "CompositionA": "20.0",
        "CompositionB": "80.0",
        "Curve": "6",
    },
]

old_bsm_iso_gradient_table = [
    {
        "Time": "Initial",
        "Flow": "0.600",
        "CompositionA": "50.0",
        "CompositionB": "50.0",
        "Curve": "Initial",
    },
    {
        "Time": "1.00",
        "Flow": "0.600",
        "CompositionA": "50.0",
        "CompositionB": "50.0",
        "Curve": "6",
    },
]

In [26]:
from OptiHPLCHandler.applications.method_converter.method_converter import (
    change_gradient_table,
)

In [27]:
# Changes current QSM gradient table to new gradient table from a BSM method
new_gradient_table = change_gradient_table(
    new_bsm_gradient_table, qsm_gradient_table_to_change
)
new_gradient_table

[{'CompositionA': '80.0',
  'CompositionB': '20.0',
  'Time': 'Initial',
  'Flow': '0.600',
  'Curve': 'Initial',
  'CompositionC': 0.0,
  'CompositionD': 0.0},
 {'CompositionA': '20.0',
  'CompositionB': '80.0',
  'Time': '1.00',
  'Flow': '0.600',
  'Curve': '6',
  'CompositionC': 0.0,
  'CompositionD': 0.0}]

In [28]:
# Changes current QSM gradient table to new gradient table from a BSM method (isocratic)
new_gradient_table = change_gradient_table(
    old_bsm_iso_gradient_table, qsm_gradient_table_to_change
)
new_gradient_table

[{'CompositionA': '50.0',
  'CompositionB': '50.0',
  'CompositionC': '0.0',
  'CompositionD': '0.0',
  'Time': 'Initial',
  'Flow': '0.600',
  'Curve': 'Initial'},
 {'CompositionA': '50.0',
  'CompositionB': '50.0',
  'CompositionC': '0.0',
  'CompositionD': '0.0',
  'Time': '1.00',
  'Flow': '0.600',
  'Curve': '6'}]

# Convert Between Detector Types
## Changing things on the instrument method level

There are two ways of changing between detector types.
1. Using channels attribute

In [31]:
bsm_pda_method_copy = bsm_pda_method.copy()
bsm_pda_method_copy.channels

[PDAChannel(wavelength1='222', wavelength2='498', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]

In [36]:
from OptiHPLCHandler.empower_detector_module_method import TUVChannel

In [35]:
tuv_method = TUVChannel(wavelength=111)
bsm_pda_method_copy.channels = [tuv_method]
bsm_pda_method_copy.channels

[PDAChannel(wavelength1='111', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]

2. Using wavelength attribute

In [37]:
bsm_pda_method_copy.wavelengths = [999, 888, 777]
bsm_pda_method_copy.wavelengths

['999', '888', '777']

In [39]:
bsm_pda_method_copy.channels

[PDAChannel(wavelength1='999', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDAChannel(wavelength1='888', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01'),
 PDAChannel(wavelength1='777', wavelength2='254', resolution='Resolution_48', datamode='DataModeAbsorbance_0', ratio2dminimumau='0.01')]